In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor
import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

from scipy.stats import norm
import numpy as np
import tensorflow as tf

In [4]:
url = "https://raw.githubusercontent.com/Tejanikhil/Shared-ASAG/main"
df = pd.read_csv(url + "/SemEvalData.csv")

In [5]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Download the stopwords corpus if not already downloaded
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def extract_pos(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    pos_words = [word for word, pos in pos_tags if pos.startswith('VB') or pos.startswith('JJ') or pos.startswith('NN')]
    return ' '.join(pos_words)

def process_dataset(text):
    lower_cased = text.lower()
    tokens = nltk.word_tokenize(lower_cased)
    filtered_tokens = [token for token in tokens if token not in stop_words]
    preprocessed = ' '.join(filtered_tokens)
    return preprocessed

In [7]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(df.loc[:,["Actual Answer","Student Answer"]], df["Grade"], test_size=0.1, random_state=42)

In [8]:
TRAIN = train_X
TRAIN["LABELS"] = train_y
TEST = test_X
TEST["LABELS"] = test_y

In [9]:
train_X["Actual Answer"] = train_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
train_X["Student Answer"] = train_X["Student Answer"].apply(extract_pos).apply(process_dataset)

test_X["Actual Answer"] = test_X["Actual Answer"].apply(extract_pos).apply(process_dataset)
test_X["Student Answer"] = test_X["Student Answer"].apply(extract_pos).apply(process_dataset)

In [10]:
print(TRAIN.shape)
TRAIN = TRAIN.replace("", pd.NA).dropna()
print(TRAIN.shape)
TEST = TEST.replace("", pd.NA).dropna()

(2197, 3)
(2184, 3)


In [11]:
train_X = TRAIN.drop("LABELS", axis = 1)
train_y = TRAIN.iloc[:,-1]
test_X = TEST.drop("LABELS", axis = 1)
test_y = TEST.iloc[:,-1]

In [12]:
sentences = list(train_X["Actual Answer"]) + list(train_X["Student Answer"])

In [13]:
rm -rf ~/.cache/huggingface/transformers/bert-base-uncased

In [14]:
from transformers import GPT2Tokenizer, GPT2Model
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [15]:
def get_gpt2_embeddings(sentence):

    input_ids = tokenizer.encode(sentence, add_special_tokens=True)
    input_ids = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(input_ids)
        embeddings = outputs[0][:, -1, :]

    return np.array(embeddings)[0]

In [16]:
train_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":train_X["Student Answer"], "Reference_Answer_Embeddings":train_X["Actual Answer"]})
test_Embeddings_df = pd.DataFrame({"Student_Answer_Embeddings":test_X["Student Answer"], "Reference_Answer_Embeddings":test_X["Actual Answer"]})

In [17]:
train_Embeddings_df

,Student_Answer_Embeddings,Reference_Answer_Embeddings
199,answer,block statement execute least
229,data functions,data members accessed member functions class d...
521,function arguments large size computing time m...,recursion step defined base case included
651,specify number elements array declaration cons...,reference
450,accessed part program referenced function foll...,makes copy function code place function call made
...,...,...
1638,list-based array easier scalable,keep rear queue pointing tail linked list enqu...
1095,nonconstant pointer nonconstant data constant ...,address location memory function code resides
1130,pointer contains address function memory,collection elements allocated
1294,need memory store previous pointers,extra space required store back pointers


In [21]:
# model = embed(sentences)

In [20]:
#get_gpt2_embeddings("Sentences")

In [18]:
train_Embeddings_df["Student_Answer_Embeddings"] = train_Embeddings_df["Student_Answer_Embeddings"].apply(get_gpt2_embeddings)
train_Embeddings_df["Reference_Answer_Embeddings"] = train_Embeddings_df["Reference_Answer_Embeddings"].apply(get_gpt2_embeddings)

test_Embeddings_df["Student_Answer_Embeddings"] = test_Embeddings_df["Student_Answer_Embeddings"].apply(get_gpt2_embeddings)
test_Embeddings_df["Reference_Answer_Embeddings"] = test_Embeddings_df["Reference_Answer_Embeddings"].apply(get_gpt2_embeddings)

In [19]:
train_Embeddings_df

,Student_Answer_Embeddings,Reference_Answer_Embeddings
199,"[-0.0574665, 0.041935623, -0.39298987, 0.02348...","[0.6590813, -0.17719454, -0.91480124, -0.21663..."
229,"[-0.26867002, -0.6126474, -1.6118188, 0.064214...","[-1.1037006, 0.12421898, -1.2950408, 0.7069706..."
521,"[-0.29278803, -0.046068694, -0.24227744, 0.038...","[0.15627639, -0.4556818, -0.5138491, 0.3930099..."
651,"[-0.5339716, 0.095146775, -0.08622505, -0.0205...","[-0.08297166, -0.0021792375, -0.5918703, -0.00..."
450,"[-0.9074699, 0.09265866, 0.52640575, 0.0758254...","[-0.47591487, -0.12559927, 0.42306903, 0.11922..."
...,...,...
1638,"[0.0002732203, -0.0072850175, -1.1602373, -0.2...","[0.49298683, 0.3126128, 0.38664925, 0.03411917..."
1095,"[0.17732099, 0.16796127, -0.28053814, -0.00620...","[-0.21876724, 0.28388837, -1.7656931, 0.047079..."
1130,"[0.095749855, -0.029838944, -0.64516747, 0.279...","[0.31974432, -0.27076206, -0.7303316, 0.007908..."
1294,"[0.061909724, -0.7052805, -1.3837694, 0.517840...","[0.048930306, -0.35124573, -1.4857486, 0.39997..."


In [20]:
train_X["Subtracted_Embeddings"] = train_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)
test_X["Subtracted_Embeddings"] = test_Embeddings_df.apply(lambda x:x["Student_Answer_Embeddings"]-x["Reference_Answer_Embeddings"], axis = 1)

In [21]:
train_df = pd.DataFrame(list(train_X["Subtracted_Embeddings"]))
test_df = pd.DataFrame(list(test_X["Subtracted_Embeddings"]))

In [25]:
train_df.to_csv("gpt2_train_embeddings_all.csv")
test_df.to_csv("gpt2_test_embeddings.csv")

In [24]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()
LR_model.fit(train_df, train_y)

LinearRegression()

In [26]:
preds = LR_model.predict(test_df)
y_preds = [round(y_val, 1) for y_val in preds]
actuals = test_y
y_true = [round(y_val, 1) for y_val in actuals]

In [27]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
LR_mae = mean_absolute_error(y_true, np.round(y_preds, 1))
LR_mse = mean_squared_error(y_true, np.round(y_preds, 1))
LR_rmse = mean_squared_error(y_true, np.round(y_preds, 1), squared=False)
LR_mape = mean_absolute_percentage_error(y_true, y_preds, multioutput = "uniform_average")
LR_r2 = r2_score(y_true, y_preds)

In [28]:
print("Mean Absolute Error (MAE): ", np.round(LR_mae, 2))
print("Mean Squared Error (MSE): ", np.round(LR_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(LR_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(LR_mape, 2))
print("R2 Score: ", np.round(LR_r2, 2))

Mean Absolute Error (MAE):  1.33
Mean Squared Error (MSE):  2.97
Root Mean Squared Error (RMSE):  1.72
Mean Absolute Percentage Error (MAPE):  395213837455209.94
R2 Score:  -0.48


In [29]:
from sklearn import neighbors
rmse_val = []
mae_val = []
mse_val = []
r2_val = []
for K in range(2,20):
    knn_model = neighbors.KNeighborsRegressor(n_neighbors = K)
    knn_model.fit(train_df, train_y)  #fit the model
    pred=knn_model.predict(test_df) #make prediction on test set
    
    mae = mean_absolute_error(y_true, np.round(pred, 1))
    mae_val.append(mae)
    
    mse = mean_squared_error(y_true, np.round(pred, 1))
    mse_val.append(mse)
    
    rmse = mean_squared_error(y_true, np.round(pred, 1), squared=False)
    rmse_val.append(rmse)
    
    r2 = r2_score(y_true, pred)
    r2_val.append(r2)

In [30]:
y_true = test_y
knn_preds = knn_model.predict(test_df)

In [31]:
min_val = min(r2_val)
index = r2_val.index(min_val)
knn_r2 = min_val
knn_rmse = rmse_val[index]
knn_mae = mae_val[index]
knn_mse = mse_val[index]

In [32]:
print("Mean Absolute Error (MAE): ", np.round(knn_mae, 2))
print("Mean Squared Error (MSE): ", np.round(knn_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(knn_rmse, 2))
print("R2 Score: ", np.round(knn_r2, 2))

Mean Absolute Error (MAE):  1.25
Mean Squared Error (MSE):  3.11
Root Mean Squared Error (RMSE):  1.76
R2 Score:  -0.54


In [33]:
ada_reg = AdaBoostRegressor(n_estimators = 100, learning_rate=0.5)

ada_reg.fit(train_df, train_y)

ada_reg_pred = ada_reg.predict(pd.DataFrame(test_df))
ada_reg_mae = mean_absolute_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mse = mean_squared_error(test_y, np.round(ada_reg_pred, 1))
ada_reg_mape = mean_absolute_percentage_error(test_y, np.round(ada_reg_pred,1), multioutput = "uniform_average")
ada_reg_rmse = mean_squared_error(test_y, np.round(ada_reg_pred, 1), squared=False)
ada_reg_r2 = r2_score(test_y, ada_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(ada_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(ada_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(ada_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(ada_reg_mape, 2))
print("R2 Score: ", np.round(ada_reg_r2, 2))

Mean Absolute Error (MAE):  1.32
Mean Squared Error (MSE):  2.21
Root Mean Squared Error (RMSE):  1.49
Mean Absolute Percentage Error (MAPE):  490800449186907.44
R2 Score:  -0.12


In [34]:
gb_reg = GradientBoostingRegressor(n_estimators = 50, max_depth=5)

gb_reg.fit(train_df, train_y)

gb_reg_pred = gb_reg.predict(pd.DataFrame(test_df))
gb_reg_mae = mean_absolute_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mse = mean_squared_error(test_y, np.round(gb_reg_pred, 1))
gb_reg_mape = mean_absolute_percentage_error(test_y, np.round(gb_reg_pred,1), multioutput = "uniform_average")
gb_reg_rmse = mean_squared_error(test_y, np.round(gb_reg_pred, 1), squared=False)
gb_reg_r2 = r2_score(test_y, gb_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(gb_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(gb_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(gb_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(gb_reg_mape, 2))
print("R2 Score: ", np.round(gb_reg_r2, 2))

Mean Absolute Error (MAE):  1.06
Mean Squared Error (MSE):  1.72
Root Mean Squared Error (RMSE):  1.31
Mean Absolute Percentage Error (MAPE):  529402731707225.94
R2 Score:  0.13


In [35]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', seed = 123)

xg_reg.fit(train_df, train_y)

# print("Best parameters: ", xg_boost_grid_search.best_params_)
# print("Best score: ", xg_boost_grid_search.best_score_)
xg_reg_pred = xg_reg.predict(pd.DataFrame(test_df))
xg_reg_mae = mean_absolute_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mse = mean_squared_error(test_y, np.round(xg_reg_pred, 1))
xg_reg_mape = mean_absolute_percentage_error(test_y, np.round(xg_reg_pred,1), multioutput = "uniform_average")
xg_reg_rmse = mean_squared_error(test_y, np.round(xg_reg_pred, 1), squared=False)
xg_reg_r2 = r2_score(test_y, xg_reg_pred)

print("Mean Absolute Error (MAE): ", np.round(xg_reg_mae, 2))
print("Mean Squared Error (MSE): ", np.round(xg_reg_mse, 2))
print("Root Mean Squared Error (RMSE): ", np.round(xg_reg_rmse, 2))
print("Mean Absolute Percentage Error (MAPE): ", np.round(xg_reg_mape, 2))
print("R2 Score: ", np.round(xg_reg_r2, 2))

Mean Absolute Error (MAE):  1.06
Mean Squared Error (MSE):  2.06
Root Mean Squared Error (RMSE):  1.44
Mean Absolute Percentage Error (MAPE):  571681414307472.5
R2 Score:  -0.04
